<a href="https://colab.research.google.com/gist/greatakela/22347b4ccf161b0fe00bb959850d6af2/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U kaggle_environments
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

     |████████████████████████████████| 1.9 MB 16.0 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 


# **Бот, выдающий только камень**

In [1]:
%%writefile rock_agent.py

#Always shows rock

def rock_always(observation, configuration):
    return 0

Writing rock_agent.py


In [4]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-98.0, 98.0]]

# **Бот, выдающий только бумагу**

In [ ]:
%%writefile paper_agent.py

#Always shows paper

def paper_always(observation, configuration):
    return 1



Writing paper_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

# **Бот, выдающий только ножницы**

In [ ]:
%%writefile scissors_agent.py

#Always shows scissors

def scissors_always(observation, configuration):
    return 2

Writing scissors_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]

# **Бот, повторяющий последний ход противника**

In [ ]:
%%writefile copy_opponent.py

import random

#Repeats opponent's last move

def copy_opponent(observation, configuration):

    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-21.0, 21.0]]

# **Бот, выдающий случайный ход**

In [17]:
%%writefile random_action.py
import random

#Shows random move

def random_action(observation, configuration):
        return random.randrange(0, configuration.signs)

Writing random_action.py


In [18]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_action.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

# **Бот, побивающий вероятный ход противника**

In [ ]:
%%writefile statistical.py
#statistically probable action 

action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Overwriting statistical.py


# **Бот, побивающий последний ход противника**

In [ ]:
%%writefile reactionary_action.py
import random

#Counters last opponent's move
last_react_action = None

def reactionary(observation, configuration):
    global last_react_action
    #initial step
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    #Counters last opponent's move
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Overwriting reactionary_action.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["reactionary",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[59.0, -59.0]]

# **Бот, побивающий последний ход противника в случае его предыдущей победы или ничьей, или побивающий его предполагаемую реакцию на мой ход в случае моей предыдущей победы**

In [ ]:
%%writefile counter_action.py
import random

#Counters last opponent's move
last_counter_action = None


def counter_reactionary(observation, configuration):
    global last_counter_action
    #initial step
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    #Counters opponent's reaction to my previous move
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    #Counters last opponent's move
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Writing counter_action.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["counter_reactionary",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[95.0, -95.0]]

# **Бот, выдающий случайный ход, не совпадающий с предыдущим ходом противника**

In [7]:
%%writefile rand_anti_opponent.py
import random

# Example 
def random_anti_opponent(observation, configuration):
    # In case we have information about opponent last move
    if observation.step > 0:
        # Full list for action
        for_action = [0,1,2]
        # Remove last action opponent
        for_action.remove(observation.lastOpponentAction)
        return random.choice(for_action)
    #initial step
    else:
         return random.randrange(0, configuration.signs)

Overwriting rand_anti_opponent.py


In [11]:
evaluate(
    "rps", #environment to use - no need to change
    ["rand_anti_opponent.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

# **Бот, выдающий ножницы на четном шаге и другие два варианта случайно на нечетном шаге**

In [28]:
%%writefile even_scissors.py
import random
# Even step scissors bot
def even_scissors(observation, configuration):
    # if even step
    if observation.step % 2 == 0: return 2
    else: return random.choice([0, 1])

Overwriting even_scissors.py


In [29]:
evaluate(
    "rps", #environment to use - no need to change
    ["even_scissors.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-27.0, 27.0]]

# **Бот, выдающий два раза бумагу, потом случайный ход**

In [12]:
%%writefile paper_x_2_rand.py
import random
# "2 x paper + random" bot
def paper_x_2_rand(observation, configuration):
    # first 2 moves
    if observation.step % 3 == 0: 
      return 1
    elif observation.step % 3 == 1:
      return 1
    # third move
    else: return random.choice([0, 2])

Writing paper_x_2_rand.py


In [14]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_x_2_rand.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-53.0, 53.0]]

# **Бот, выдающий два раза камень, потом два случайных хода**

In [15]:
%%writefile rock_x_2_rand_x_2.py
import random
# "2 x rock + 2 x random" bot
def rock_x_2_rand_x_2(observation, configuration):
    # first 2 moves
    if observation.step % 4 in [0,1]: 
      return 0
    # next 2 moves
    else: return random.choice([1, 2])

Writing rock_x_2_rand_x_2.py


In [16]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_x_2_rand_x_2.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-20.0, 20.0]]

# **Бот, повторяющий последний ход противника на нечетном шаге и побивающий предыдущий ход противника на четном шаге** 

In [19]:
%%writefile odd_copy_opponent.py
import random

# copies last opponent's move on the odd step, counters last opponent's move on the even step

def odd_copy_opponent(observation, configuration):
    # In case we have information about opponent last move
    if observation.step > 0:
        # odd step
        if observation.step % 2 == 1: 
            return observation.lastOpponentAction      
        # even step
        else:
            return (observation.lastOpponentAction + 1) % configuration.signs
    # Initial step
    else:
         return random.randrange(0, configuration.signs)



Writing odd_copy_opponent.py


In [20]:
evaluate(
    "rps", #environment to use - no need to change
    ["odd_copy_opponent.py",  "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[94.0, -94.0]]

# **Проведение турнира**

In [30]:
def get_pairs(lst, n):
    #функция разбиения списка список 
    #списков по n элементов
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def round_tournaments(players):
    
    #функция гругового турнира players
   
    qty_players = len(players)
    scores = [0 for _ in range(qty_players)]
    for x in range(qty_players):
      for y in range(x + 1, qty_players):
        for z in range(PERIODS):
            res = evaluate("rps", [players[x], players[y]],
                            configuration = {
                                           'episodeSteps': 100,
                                           'tieRewardThreshold':1
                                           }
                            )[0]
            if not period: results.append(res)
            else:
              scores[players.index(players[x])] += res[0]
              scores[players.index(players[y])] += res[1]
    sort_scores = sorted(scores)          
    players = [players[scores.index(score)] for score in sort_scores]
    return players[qty_players // 2:]

players = ['rock',
           'paper',
           'scissors',
           'copy_opponent',
           'reactionary',
           'counter_reactionary',
           'paper_x_2_rand.py',
           'odd_copy_opponent.py',
           'rock_x_2_rand_x_2.py',
           'even_scissors.py',
           'rand_anti_opponent.py',
           'random_action.py'
]

# периодов в парной игре
PERIODS = 5
# перемешиваем участников
random.shuffle(players)

qty_players = len(players)
PLAY = True
step = 1

while PLAY:
    if qty_players % 2:
      print('Нечетное количество участников - нужен круговой турнир')
      print('______________________')
      # Собираем победителей кругового турнира функцией
      players = round_tournaments(players)

    else:
        pairs = qty_players // 2

        if not pairs % 2:
            print('Турнир пар по жребию')
            print('______________________')
            groups = list(get_pairs(players, 2))
            print(f'Количество турнирных пар {len(groups)}')
            results = []
            for period in range(PERIODS):
              for i, pair in enumerate(groups):
                #print(pair)
                res = evaluate(
                "rps", #environment to use - no need to change
                pair, #agents to evaluate
                configuration={
                              'episodeSteps': 100,
                              'tieRewardThreshold':1
                               } #number of episodes 
                )[0]
                #print(res)
                if not period: results.append(res)
                else:
                  results[i][0] += res[0]
                  results[i][1] += res[1]
              # Собираем победителей этапа из победителей пар
              players = []
              for result, pair in zip(results, groups):
                  if result[0]  >result[1]: players.append(pair[0])
                  elif result[0] < result[1]: players.append(pair[1])
                  # Берем обоих если нет победителя
                  else: players.extend(pair)

        else:
          
          print('Нечетное количество пар - нужен круговой турнир')
          print('______________________')
          # Собираем победителей кругового турнира функцией
          players = round_tournaments(players)


    print(f'Победители этапа {step}')
    for player in players:
      print(player)
    print()
    # перемешиваем участников
    random.shuffle(players)
    step += 1

    qty_players = len(players)
    if qty_players < 2:
      print(f'Победитель {players[0]}')
      PLAY = False

Турнир пар по жребию
______________________
Количество турнирных пар 6
Победители этапа 1
even_scissors.py
paper
copy_opponent
reactionary
rock_x_2_rand_x_2.py
rand_anti_opponent.py

Нечетное количество пар - нужен круговой турнир
______________________
Победители этапа 2
rand_anti_opponent.py
even_scissors.py
reactionary

Нечетное количество участников - нужен круговой турнир
______________________
Победители этапа 3
rand_anti_opponent.py
even_scissors.py

Нечетное количество пар - нужен круговой турнир
______________________
Победители этапа 4
even_scissors.py

Победитель even_scissors.py
